In [ ]:
import pandas as pd
import numpy as np
ruta='/content/drive/MyDrive/Colab Notebooks/ITESO/Ingeniería de características/P2022_MCD3735A/Módulo 4 - Codificación de variables categóricas/'

data=pd.read_csv(ruta+'temps.txt',sep='\s+',header=None) # Fuente: http://astro.iam.udg.mx/Estacion/NOAAPRMO.TXT
                                                         #         http://astro.iam.udg.mx/Estacion/NOAAMO.TXT
data.columns=['Temperatura máxima','Temperatura mínima','HDD','CDD','Rain']
d=['Sábado','Domingo','Lunes','Martes','Miércoles','Jueves','Viernes']
dias=[d[i%7] for i in range(len(data))]
catego=['Soleado','Mayormente soleado','Parcialmente nublado','Mayormente nublado','Nublado','Lluvioso']
prono=[catego[-1] if data['Rain'][i]>0 else catego[int(round(1.2*(data['HDD'][i]-data['CDD'][i])+2))] for i in range(len(data))]
df=pd.DataFrame(columns=['Días','Pronóstico','Temperatura máxima','Temperatura mínima'],index=range(len(data)))
df.iloc[:,2:4]=data.iloc[:,0:2]
df['Días']=dias
df['Pronóstico']=prono
df

In [2]:
catego=df['Días'].unique()
catego

array(['Sábado', 'Domingo', 'Lunes', 'Martes', 'Miércoles', 'Jueves',
       'Viernes'], dtype=object)

In [3]:
# Cardinalidad
n=len(catego)
n

7

In [4]:
# Ordenamos las categorías
cat_ord=np.concatenate([catego[1:],np.array([catego[0]])])
cat_ord

array(['Domingo', 'Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes',
       'Sábado'], dtype=object)

Una vez ordenadas las categorías, asignamos los índices como:
$$
x=\left\{\begin{array}{ll}
\{-(n-1):2:n-1\}& n=par\\
\left\{-\frac{n-1}{2}:1:\frac{n-1}{2}\right\}&n=impar
\end{array}\right.
$$

In [5]:
bg=n-1 if n%2==0 else (n-1)//2
st=2 if n%2==0 else 1

x=range(-bg,bg+1,st)
for i in x:
  print(i)

-3
-2
-1
0
1
2
3


Codificación de la variable `Días`

In [6]:
# Polinomio lineal
#
m=1
d1={c:m*i for c,i in zip(cat_ord,x)}
d1

{'Domingo': -3,
 'Jueves': 1,
 'Lunes': -2,
 'Martes': -1,
 'Miércoles': 0,
 'Sábado': 3,
 'Viernes': 2}

In [ ]:
# Mapeo
dfC=df.copy()
dfC['Días_Lin']=df['Días'].map(d1)
dfC

Asignando $a_1=0$, se debe cumplir que
$$
-na_0=a_2\sum_{i=x}i^2,
$$
donde $n$ es la cardinalidad de la variable y $x$ son los índices que expresan el órden de las categorías.

In [8]:
# Polinomio cuadrático
#
a1=0
a0=1 # Asignación (Cóncavo)
a2=-n*a0/sum([i**2 for i in x])
d2={c:a0+a1*i+a2*i**2 for c,i in zip(cat_ord,x)}
d2

{'Domingo': -1.25,
 'Jueves': 0.75,
 'Lunes': 0.0,
 'Martes': 0.75,
 'Miércoles': 1.0,
 'Sábado': -1.25,
 'Viernes': 0.0}

In [ ]:
dfC['Días_Cuad']=df['Días'].map(d2)
dfC

In [10]:
m=0
for v in d1.values():
  m += v**2
m # Magnitud al cuadrado

28

In [11]:
# Normalizamos para asignarle el mismo peso a la codificación polinomial, tanto lineal como cuadrática
d1n=d1.copy()
for c,v in d1.items():
  d1n[c]=v/np.sqrt(m)
d1n

{'Domingo': -0.5669467095138409,
 'Jueves': 0.1889822365046136,
 'Lunes': -0.3779644730092272,
 'Martes': -0.1889822365046136,
 'Miércoles': 0.0,
 'Sábado': 0.5669467095138409,
 'Viernes': 0.3779644730092272}

In [12]:
m=0
for v in d2.values():
  m += v**2
m

5.25

In [13]:
d2n=d2.copy()
for c,v in d2.items():
  d2n[c]=v/np.sqrt(m)
d2n

{'Domingo': -0.545544725589981,
 'Jueves': 0.3273268353539886,
 'Lunes': 0.0,
 'Martes': 0.3273268353539886,
 'Miércoles': 0.4364357804719848,
 'Sábado': -0.545544725589981,
 'Viernes': 0.0}

In [14]:
m=0
for v in d1n.values():
  m += v**2
m

1.0

In [15]:
m=0
for v in d2n.values():
  m += v**2
m

1.0000000000000002

In [19]:
dfC['Días_Lin']=df['Días'].map(d1n)
dfC['Días_Cuad']=df['Días'].map(d2n)
dfC.head()

,Días,Pronóstico,Temperatura máxima,Temperatura mínima,Días_Lin,Días_Cuad
0,Sábado,Parcialmente nublado,28.2,11.2,0.566947,-0.545545
1,Domingo,Parcialmente nublado,29.1,10.7,-0.566947,-0.545545
2,Lunes,Mayormente nublado,26.1,10.4,-0.377964,0.000000
3,Martes,Mayormente soleado,29.4,11.1,-0.188982,0.327327
4,Miércoles,Mayormente soleado,29.8,11.1,0.000000,0.436436


Codificación para la variable `Pronóstico`

In [17]:
catego=df['Pronóstico'].unique()
catego

array(['Parcialmente nublado', 'Mayormente nublado', 'Mayormente soleado',
       'Soleado', 'Lluvioso', 'Nublado'], dtype=object)

In [18]:
n=len(catego)
n

6

In [20]:
# Definimos la variable independiente 'x'
bg=n-1 if n%2==0 else (n-1)//2
st=2 if n%2==0 else 1
x=range(-bg,bg+1,st)
for i in x:
  print(i)

-5
-3
-1
1
3
5


In [21]:
ord=[catego[3],catego[2],catego[0],catego[1],catego[-1],catego[-2]]
ord

['Soleado',
 'Mayormente soleado',
 'Parcialmente nublado',
 'Mayormente nublado',
 'Nublado',
 'Lluvioso']

In [31]:
# Codificación cuadrática
a0=-1
a1=0
a2=-n*a0/sum([i**2 for i in x])
p2={c:a0+a1*i+a2*i**2 for c,i in zip(ord,x)}
p2

{'Lluvioso': 1.1428571428571428,
 'Mayormente nublado': -0.9142857142857143,
 'Mayormente soleado': -0.22857142857142854,
 'Nublado': -0.22857142857142854,
 'Parcialmente nublado': -0.9142857142857143,
 'Soleado': 1.1428571428571428}

In [32]:
suma=0
for v in p2.values():
  suma+=v
suma

0.0

In [33]:
norma=0
for v in p2.values():
  norma+=v**2
norma # Cuadrado de la norma euclideana

4.388571428571428

In [34]:
p2n=p2.copy()
for c,v in p1.items():
  p2n[c]=v/np.sqrt(norma)
p2n

{'Lluvioso': 0.5455447255899809,
 'Mayormente nublado': -0.4364357804719848,
 'Mayormente soleado': -0.10910894511799618,
 'Nublado': -0.10910894511799618,
 'Parcialmente nublado': -0.4364357804719848,
 'Soleado': 0.5455447255899809}

In [35]:
norma=0
for v in p2n.values():
  norma+=v**2
norma # Cuadrado de la norma euclideana

0.9999999999999999

In [36]:
dfC['Pronóstico_Cuad']=df['Pronóstico'].map(p2n)
dfC.head()

,Días,Pronóstico,Temperatura máxima,Temperatura mínima,Días_Lin,Días_Cuad,Pronóstico_Lin,Pronóstico_Cuad
0,Sábado,Parcialmente nublado,28.2,11.2,0.566947,-0.545545,-0.436436,-0.436436
1,Domingo,Parcialmente nublado,29.1,10.7,-0.566947,-0.545545,-0.436436,-0.436436
2,Lunes,Mayormente nublado,26.1,10.4,-0.377964,0.000000,-0.436436,-0.436436
3,Martes,Mayormente soleado,29.4,11.1,-0.188982,0.327327,-0.109109,-0.109109
4,Miércoles,Mayormente soleado,29.8,11.1,0.000000,0.436436,-0.109109,-0.109109


In [37]:
for i in x:
  print(i)

-5
-3
-1
1
3
5


In [38]:
# Codificación lineal
m=1/np.sqrt(70)
p1={c:m*i for c,i in zip(ord,x)}
p1

{'Lluvioso': 0.5976143046671968,
 'Mayormente nublado': 0.11952286093343936,
 'Mayormente soleado': -0.3585685828003181,
 'Nublado': 0.3585685828003181,
 'Parcialmente nublado': -0.11952286093343936,
 'Soleado': -0.5976143046671968}

In [39]:
norma=0
for v in p1.values():
  norma+=v**2
norma

0.9999999999999999

In [40]:
dfC['Pronóstico_Lin']=df['Pronóstico'].map(p1)
dfC.head()

,Días,Pronóstico,Temperatura máxima,Temperatura mínima,Días_Lin,Días_Cuad,Pronóstico_Lin,Pronóstico_Cuad
0,Sábado,Parcialmente nublado,28.2,11.2,0.566947,-0.545545,-0.119523,-0.436436
1,Domingo,Parcialmente nublado,29.1,10.7,-0.566947,-0.545545,-0.119523,-0.436436
2,Lunes,Mayormente nublado,26.1,10.4,-0.377964,0.000000,0.119523,-0.436436
3,Martes,Mayormente soleado,29.4,11.1,-0.188982,0.327327,-0.358569,-0.109109
4,Miércoles,Mayormente soleado,29.8,11.1,0.000000,0.436436,-0.358569,-0.109109
